In [1]:
import pandas as pd
import numpy as np
import pickle
import json

In [2]:
# import filtered business_id as dataframe from pickle
with open('data/yelp-dataset/df_business.pickle', 'rb') as fp:
    df_business_id = pickle.load(fp)
print(df_business_id.name)
df_business_id.shape

business_id


(38117,)

In [3]:
# readin tip.json
with open('data/yelp-dataset/yelp_academic_dataset_tip.json') as fp:
    json_tip = fp.readlines()
json_tip[:1]

['{"text":"Get here early enough to have dinner.","date":"2012-07-15","likes":0,"business_id":"tJRDll5yqpZwehenzE2cSg","user_id":"zcTZk7OG8ovAmh_fenH21g"}\n']

In [4]:
# Create dataframe and filter by cities
list_ = []
for i_line in json_tip:
    list_.append(json.loads(i_line))
df_tip = pd.DataFrame(list_)
print('before fileter', df_tip.shape)
print('no NaN, as show in isna():', df_tip.isna().sum())
df_tip = df_tip[df_tip['business_id'].isin(df_business_id)]
print('after filter', df_tip.shape)
df_tip.head()
    

before fileter (1185348, 5)
no NaN, as show in isna(): business_id    0
date           0
likes          0
text           0
user_id        0
dtype: int64
after filter (472976, 5)


,business_id,date,likes,text,user_id
53,QFPS9JBNdOOHVP0C3pSslw,2012-08-22,0,"The place to go after work, but, it ain't cheap",TvkqJ8YEIsTb16RnnrNyfQ
131,MTsIckdo3_uKuqk3B4zuKA,2012-10-20,0,The Cleveland Pickle is the best sandwich deli...,MolXvMRbUNY6Yr_s0zEQ0A
216,gh6__q2WXFuyN8gt6VAnWw,2016-04-11,0,"Pittsburgh says: GREAT FOOD! Thanks, Luby!",PZNMPWCViVX8JLsn10MSnQ
399,uv-e-d43NN-1_DcCoJL-Og,2011-11-26,0,Christmas doing for Judee do.e,blrWvPePSv87aU9hV1Zd8Q
401,0youcKV6-eE3F2MQj1l6Fw,2010-11-25,0,A bit different. Out at a restaurant for Thank...,blrWvPePSv87aU9hV1Zd8Q
